In [11]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d johnsmith88/heart-disease-dataset
!unzip \*.zip && rm *.zip

  0% 0.00/6.18k [00:00<?, ?B/s]
100% 6.18k/6.18k [00:00<00:00, 7.85MB/s]
Archive:  heart-disease-dataset.zip
  inflating: heart.csv               


In [17]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [18]:
# Load the dataset
data = pd.read_csv('/content/heart.csv')

In [19]:
# Handle missing values
data = data.dropna()


In [20]:
# Convert categorical variables to numeric
data['target'] = data['target'].map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})

In [21]:
# Select features for analysis
features = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
            'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

In [22]:
X = data[features]
y = data['target']

In [23]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
# Reset indices after splitting
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [25]:
# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [26]:
# Define a function to calculate fuzzy membership values
def calculate_membership(x, cluster_centers, m):
    distances = np.linalg.norm(x - cluster_centers, axis=1)
    membership_values = 1.0 / np.sum((distances / distances[:, np.newaxis]) ** (2 / (m - 1)), axis=1)
    return membership_values

In [29]:
# Define a function to perform Fuzzy kNN classification
def fuzzy_knn(X_train, y_train, X_test, k, c, m):
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    y_pred = np.zeros(n_test)

    for i in range(n_test):
        distances = np.linalg.norm(X_train - X_test[i], axis=1)
        sorted_indices = np.argsort(distances)
        k_nearest_indices = sorted_indices[:k]
        k_nearest_labels = y_train[k_nearest_indices]

        cluster_centers = np.zeros((c, X_train.shape[1]))
        for j in range(c):
            cluster_centers[j] = np.mean(X_train[y_train == j][k_nearest_indices], axis=0)

        membership_values = calculate_membership(X_test[i], cluster_centers, m)
        y_pred[i] = np.argmax(np.bincount(k_nearest_labels, weights=membership_values))

    return y_pred.astype(int)

In [32]:
# Parameters
k = 3  # Number of neighbors
c = 2  # Number of clusters
m = 2  # Degree of fuzziness


In [36]:
!pip install scikit-learn

In [37]:
from sklearn.neighbors import NearestNeighbors

In [42]:
# Fuzzy kNN classification
neigh = NearestNeighbors(n_neighbors=k)
neigh.fit(X_train)
distances, indices = neigh.kneighbors(X_test)


In [39]:
print(X_train.shape)
print(X_test.shape)

(820, 13)
(205, 13)


In [40]:
print(X_train.ndim)
print(X_test.ndim)

2
2


In [41]:
from scipy.spatial.distance import cdist

distances = cdist(X_train, X_test)

In [43]:
y_pred = fuzzy_knn(X_train, y_train, indices, k, c, m)

ValueError: operands could not be broadcast together with shapes (820,13) (3,) 

In [44]:
print(X_train.shape)
print(X_test.shape)

(820, 13)
(205, 13)


In [48]:
import numpy as np

In [49]:
print(X_test.shape)

(205, 13)


In [50]:
num_elements = np.prod(X_test.shape)

In [51]:
expected_shape = (X_test.shape[0], 3, 1)
num_expected_elements = np.prod(expected_shape)

In [52]:
if num_elements != num_expected_elements:
    print("Cannot reshape array: incompatible number of elements.")
else:
    # Reshape the array
    X_test = X_test.reshape(expected_shape)

Cannot reshape array: incompatible number of elements.


In [53]:
print(X_test.shape)

(205, 13)


In [54]:
y_pred = fuzzy_knn(X_train, y_train, indices, k, c, m)

ValueError: operands could not be broadcast together with shapes (820,13) (3,) 